In [1]:
import pickle
import random
import string
import time
import pandas as pd

# Nuevos usuarios

Vamos a asociar un nombre de usuario y contraseña a cada uno de los userIds existentes en el dataset de ratings

In [ ]:
def create_user_password_dict(num_users):
    users = {}
    for i in range(num_users):
        user = f"user{i + 1}"
        password = ''.join(random.choices(string.ascii_letters + string.digits, k=12))
        users[i + 1] = {"user": user, "password": password}
    return users

num_users = 610
user_password_dict = create_user_password_dict(num_users)

# Show users' information
for user_id, data in user_password_dict.items():
    print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")


In [ ]:
# Change the password of users 1 and 4
user_password_dict[1]["password"] = "123"
user_password_dict[4]["password"] = "456"

# # Show users' information
# for user_id, data in user_password.items():
#     print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")

In [ ]:
# Save the dictionary
with open("user_password_dict.pkl", 'wb') as file:
        pickle.dump(user_password_dict, file)

Añadir nuevos usuarios a la base de datos

In [ ]:
def add_new_user(username, password, users):
    for data in users.values():
        if data["user"] == username:
            print(f"El nombre de usuario '{username}' ya está en uso.")
            return

    # Obtener el último ID existente y aumentarlo en 1
    last_id = max(users.keys())
    new_id = last_id + 1

    # Agregar el nuevo usuario al diccionario
    users[new_id] = {"user": username, "password": password}
    print(f"Nuevo usuario agregado: ID: {new_id} | User: {username} | Password: {password}")

# Ejemplo de uso
username = input("Introduce el nombre de usuario: ")
password = input("Introduce la contraseña: ")

add_new_user(username, password, user_password_dict)

In [ ]:
user_password_dict

In [ ]:
existing_usernames = [user['user'] for user in user_password_dict.values()]
existing_usernames

In [ ]:
def existing_username(username,user_password_dict):
    return username in [user['user'] for user in user_password_dict.values()]

existing_username("user5555",user_password_dict)

In [ ]:
# Load user-password database
with open('user_password_dict.pkl', 'rb') as file:
    user_password_dict = pickle.load(file)

In [ ]:
user_password_dict

# QUIZ. Obtener las películas con mayor número de valoraciones

Las 15 películas más valoradas se mostrarán a un nuevo usuario tras registrarse con el fin de conocer de forma superficial sus gustos cinematográficos y poder realizar recomendaciones personalizadas

In [2]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()
ratings = ml.ratings.copy()

In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
k = 15
top_movies = ratings.groupby('movieId').size().nlargest(k).index.tolist()
top_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in top_movies]
top_movies_titles

['Forrest Gump',
 'Shawshank Redemption, The',
 'Pulp Fiction',
 'Silence of the Lambs, The',
 'Matrix, The',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Usual Suspects, The',
 'American Beauty']

Vamos a adaptar el nombre de las películas para que sea más sencillo de leer

In [5]:
top_movies_titles = ['Forrest Gump',
 'The Shawshank Redemption',
 'Pulp Fiction',
 'The Silence of the Lambs',
 'The Matrix',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V',
 'The Usual Suspects',
 'American Beauty']

In [6]:
top_movies_dict = {}
for movie_id, movie_title in zip(top_movies, top_movies_titles):
    top_movies_dict[movie_id] = movie_title
    
top_movies_dict

{356: 'Forrest Gump',
 318: 'The Shawshank Redemption',
 296: 'Pulp Fiction',
 593: 'The Silence of the Lambs',
 2571: 'The Matrix',
 260: 'Star Wars: Episode IV - A New Hope',
 480: 'Jurassic Park',
 110: 'Braveheart',
 589: 'Terminator 2: Judgment Day',
 527: "Schindler's List",
 2959: 'Fight Club',
 1: 'Toy Story',
 1196: 'Star Wars: Episode V',
 50: 'The Usual Suspects',
 2858: 'American Beauty'}

In [7]:
# Save the dictionary
with open("../dashboard/top_movies_dict.pkl", 'wb') as file:
        pickle.dump(top_movies_dict, file)

# Recomendaciones

In [ ]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()
name_to_movieID = ml.name_to_movieID.copy()

with open("../dashboard/name_to_movieID.pkl", 'wb') as file:
        pickle.dump(name_to_movieID, file)

Vamos a ver cuales son las películas mejor valoradas. Para ello primero seleccionamos las 100 películas con mayor número de valoraciones, y entre ellas se seleccionan aquellas con mejor valoración media

In [32]:
most_rated_movies = ratings.groupby('movieId').count().sort_values(by='rating', ascending=False).nlargest(100, 'rating')
most_rated_movies = most_rated_movies.reset_index()
ids_most_rated_movies = most_rated_movies['movieId']
ids_most_rated_movies

# Filter the ratings dataframe to obtain the information about the most rated mvoies
filtered_df = ratings[ratings['movieId'].isin(ids_most_rated_movies)]

# Compute the average rating of the movies
sorted_ratings = filtered_df.groupby('movieId').mean('rating').sort_values(by='rating', ascending=False).nlargest(10, 'rating')
sorted_ratings = sorted_ratings.reset_index()
sorted_movies = sorted_ratings['movieId']


sorted_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in sorted_movies]
sorted_movies_titles

sorted_movies_dict = {}
for movie_id, movie_title in zip(sorted_movies, sorted_movies_titles):
    sorted_movies_dict[movie_id] = movie_title
    
sorted_movies_dict

{318: 'Shawshank Redemption, The',
 858: 'Godfather, The',
 2959: 'Fight Club',
 1221: 'Godfather: Part II, The',
 1213: 'Goodfellas',
 58559: 'Dark Knight, The',
 50: 'Usual Suspects, The',
 1197: 'Princess Bride, The',
 260: 'Star Wars: Episode IV - A New Hope',
 527: "Schindler's List"}

In [33]:
# Common elements in top_movies and sorted_movies
list(set(top_movies_titles) & set(sorted_movies_titles))

['Fight Club', "Schindler's List", 'Star Wars: Episode IV - A New Hope']

Las dos películas más populares ('The Shawshank Redemption' and 'The Godfather) se recomendarán a todos los usuarios nuevos, dado que son populares entre los usuarios actuales del sistema y no se dispone información acerca de si han visto o no la película. Por otro lado, para el caso de las películas 'Fight Club', 'Schindler's List' y'Star Wars: Episode IV - A New Hope' se procede de diferente manera. Como estas películas se incluyen en el cuestionario inicial que debe rellenar el usuario solo se recomendarán en el caso de que el usuario las marque como no vistas. El resto de recomendaciones hasta llegar a 10 se obtienen calculando la distancia entre usuarios y seleccionando aquellas recomendaciones que se hagan al usuario con gustos más semejantes al nuevo usuario.

**¿Esta opción o mejor recomiendo las películas que más le hayan gustado al usuario parecido?**

## Cálculo de la distancia entre usuarios

En primer lugar vamos a añadir la información del nuevo usuario al conjunto de datos existente

In [52]:
# Load new users' info
with open('../dashboard/valores_slider.pkl', 'rb') as file:
    valores_slider = pickle.load(file)
    
valores_slider

{356: 0,
 318: 0,
 296: 0,
 593: 0,
 2571: 0,
 260: 0,
 480: 0,
 110: 0,
 589: 0,
 527: 0,
 2959: 5,
 1: 1,
 1196: 0,
 50: 0,
 2858: 0}

In [39]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [68]:
userId = ratings['userId'].max()+1
timestamp = int(time.time())

# Add the slider information to the ratings dataframe
new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
new_ratings.index.name = 'movieId'
new_ratings = new_ratings.reset_index()
new_ratings['userId'] = userId
new_ratings['timestamp'] = timestamp

# Reorder the columns in `df_new_ratings` to match the order of the existing DataFrame
new_ratings = new_ratings.reindex(columns=ratings.columns)
new_ratings

,userId,movieId,rating,timestamp
0,612,356,0,1688576243
1,612,318,0,1688576243
2,612,296,0,1688576243
3,612,593,0,1688576243
4,612,2571,0,1688576243
5,612,260,0,1688576243
6,612,480,0,1688576243
7,612,110,0,1688576243
8,612,589,0,1688576243
9,612,527,0,1688576243


In [ ]:
# Concatenate the `ratings` DataFrame and `df_new_ratings` to add the new lines at the end
ratings = pd.concat([ratings, new_ratings], ignore_index=True)
ratings

In [74]:
def add_new_user():
    # Load new users' info
    with open('../dashboard/valores_slider.pkl', 'rb') as file:
        valores_slider = pickle.load(file)
        
    #ratings = pd.read_csv('ratings.csv')
    
    userId = ratings['userId'].max()+1
    timestamp = int(time.time())
    
    

    # Add the slider information to the ratings dataframe
    new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
    new_ratings.index.name = 'movieId'
    new_ratings = new_ratings.reset_index()
    new_ratings['userId'] = userId
    new_ratings['timestamp'] = timestamp

    # Reorder the columns in new_ratings to match the order of the ratings DataFrame
    new_ratings = new_ratings.reindex(columns=ratings.columns)
    
    # Concatenate the ratings DataFrame and new_ratings to add the new lines at the end
    ratings = pd.concat([ratings, new_ratings], ignore_index=True)
    
    return ratings
    
    # Save the updated file
    # ratings.to_csv('ratings.csv', index=False)

Para ello vamos a aprovechar el código generado para el algoritmo de filtrado colaborativo basado en usuarios https://github.com/inesgfortis/RecommenderSys/blob/main/collaborativeFiltering/userBased.ipynb

# Recomendaciones usuarios existentes

In [77]:
with open('../dashboard/recommendations.pkl', 'rb') as file:
        recommendations = pickle.load(file)
recommendations

defaultdict(list,
            {432: [(177593, 4.536487157083084),
              (4306, 4.515574053873634),
              (1172, 4.508916504810848),
              (2318, 4.465865536206673),
              (62, 4.451060959559145),
              (3552, 4.4396649773210175),
              (362, 4.4264878141282935),
              (3147, 4.418520016587123),
              (3429, 4.374524305135452),
              (2078, 4.366759349542657)],
             288: [(1221, 4.458994959514339),
              (4226, 4.439926453925377),
              (720, 4.364443450137575),
              (2858, 4.357646031799315),
              (3996, 4.352513707316535),
              (4973, 4.347870320861219),
              (3435, 4.346342921681236),
              (858, 4.307949931502534),
              (50, 4.299046384254374),
              (1178, 4.298004244542662)],
             599: [(260, 4.3274964294039115),
              (527, 4.016548175623),
              (904, 3.9891932804847157),
              (1203, 3.977147

In [80]:
def get_movie_images(userId):

    numbers = [item[0] for item in recommendations[userId]]
    images = []
    for number in numbers:
        image_path = f"{number}.jpg"
        image = html.Img(src=dash.get_asset_url(image_path), style={"width": "110px", "height": "140px", "margin": "10px"})
        images.append(image)
    return images


177593

In [82]:
numbers = [item[0] for item in recommendations[432]]
type(numbers)

list

In [108]:
def get_user_preferences(userId, k, like=True):

    # Filter ratings for the specified userId and sort movies by rating in descending order
    user_ratings = ratings[ratings['userId'] == userId].copy()
    sorted_movies = user_ratings.sort_values(by='rating', ascending=False)

    if like:
        # Get the top k movies with highest ratings
        movieIds = list(sorted_movies.head(k).copy()['movieId'])
    
    # Get the top k movies with lowest ratings
    else:
        movieIds = list(sorted_movies.tail(k).copy()['movieId'])

    return movieIds

In [109]:
numbers = get_user_preferences(1, 10, False)
numbers

[1030, 1009, 780, 3243, 1219, 2617, 2253, 2338, 2389, 3176]